<a href="https://colab.research.google.com/github/mirrormouse/machine_learning/blob/main/GAN_english_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

アルファベットと数字の変換準備

In [1]:
char="abcdefghijklmnopqrstuvwxyz"
dic={}
for i in range(len(char)):
  dic[char[i]]=i
print(dic['l'])

11


ドライブをインポート

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


でたらめな５文字からなる「ランダム文字列」を生成。これを上のcharに対応させて数値に変換
例）["abcdz",...]->[[0,1,2,3,25],...]

In [3]:
import random
def generate(data_size=1000,length=5):
  random_list=[]
  random_words=[]
  for i in range(data_size):
    word=''
    num=[]
    for j in range(length):
      a=random.randrange(26)
      c=char[a]
      word+=c
      num.append(a)
    random_list.append(num)
    random_words.append(word)
  return random_list,random_words

lists,word=generate()

英文データの読み込み

In [4]:
base='drive/MyDrive/ML/train/'
f = open(base+'train.txt', 'r')
rawdata = f.read()
data=rawdata.split()
print(data[:5])
f.close()

['The', 'two', 'submarines', 'of', 'the']


５文字の長さの英単語だけを大量に抽出し、「存在する英単語」としてまとめる。これも数値に変換。

In [5]:
import re
def correct_generate(rawdata,data_size=10000,length=5):
  alpha_data=[]
  for word in rawdata:
    word=word.lower()
    res = re.sub(r'[^a-zA-Z]', '', word)
    if len(res)==length:
      alpha_data.append(res)
  set_alpha=list(set(alpha_data))

  beta_data=[]
  for word in set_alpha:
    num=[]
    for id in range(len(word)):
      c=word[id]
      num.append(dic[c])
    beta_data.append(num)
  size=len(beta_data)
  print(size)
  return set_alpha, beta_data, size

alpha,beta,_=correct_generate(data)
print(beta[:5])

961
[[1, 0, 18, 4, 3], [2, 11, 4, 0, 17], [8, 13, 4, 17, 19], [0, 6, 4, 13, 19], [20, 15, 15, 4, 17]]


訓練データの生成（上で作った２つを繋げるだけ）

In [6]:
x_train_rawlist=beta
y_train_list=[]
#for i in range(len(lists)):
#  y_train_list.append(0)
for i in range(len(beta)):
  y_train_list.append(1)
print(x_train_rawlist[0])

[1, 0, 18, 4, 3]


訓練データのリストをnumpyに変換。正解ラベルを生成

In [7]:
import numpy as np
x_train_list=[]
for data in x_train_rawlist:
  item=[]
  for num in data:
    one_hot=np.zeros(26)
    one_hot[num]=1
    one_hot_list=one_hot.tolist()
    item.append(one_hot_list)
  x_train_list.append(item)
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
print(x_train[0])
print(y_train[-5:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
[1 1 1 1 1]


In [8]:
print(x_train.shape)
print(y_train.shape)

(961, 5, 26)
(961,)


In [9]:
def word2num(word):
  num=[]
  for id in range(len(word)):
    num.append(dic[word[id]])
  return num
def num2word(nums):
  word=''
  for i in nums:
    word+=char[i]
  return word
print(num2word([1,4, 0, 18, 19]))
print(word2num("beast"))

beast
[1, 4, 0, 18, 19]


単語とベクトルの変換用関数

In [10]:
def word2num(word):
  num=[]
  for id in range(len(word)):
    num.append(dic[word[id]])
  return num
def num2word(nums):
  word=''
  for i in nums:
    word+=char[i]
  return word
print(num2word([1,4, 0, 18, 19]))
print(word2num("beast"))

beast
[1, 4, 0, 18, 19]


In [11]:
def nothing_val(word):#元データ内にその単語が存在するか確認
  for w in alpha:
    if word==w:
      return False
  return True
nothing_val('joker')
nothing_val('beast')
nothing_val('criry')

True

In [12]:
test_words=["beast","blade","joker","kdjus","adjsa","flakq"]
y_test=[[0,1],[0,1],[0,1],[1,0],[1,0],[1,0]]
x_test=[]
for word in test_words:
  vec=word2num(word)
  item=[]
  for num in vec:
    data=np.zeros(26,dtype=int)
    data[num]=1
    data_list=data.tolist()
    item.append(data_list)
  x_test.append(item)

In [14]:
import numpy as np
import math
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.models import Sequential
#from keras.optimizers import Adam

#mnistの形状[28, 28, 1]を定義

#generatorが画像を生成するために入力させてあげるノイズの次元
z_dim = 100

#生成器
def create_generator(shape,all,dim):
  model = Sequential()
  model.add(Dense(128, input_dim=dim))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(Dense(128, input_dim=dim))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(Dense(128, input_dim=dim))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(Dense(128, input_dim=dim))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(Dense(all, activation='tanh'))
  model.add(Reshape(shape))
  return model

#識別器
def create_discriminatior(shape):
  model = Sequential()
  model.add(Flatten(input_shape=shape))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(Dense(2, activation='sigmoid'))
  return model

#Ganのモデル定義する
def create_gan(generator, discriminator):
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  return model


def convert(words):
  res=""
  for vec in words:
    res+=char[np.argmax(np.array(vec))]
  return res


#学習させてあげるための関数。イテレーション数、バッチサイズ、　何イテレーションで画像を生成して可視化するかを引数にとる
def train(x_train,shape ,all , dim, optimizer, batch_size, sample_interval,iterations ):
#学習の記録用
  loss_m=[]
  acc_m=[]
#各モデルの生成とコンパイル
  discriminator = create_discriminatior(shape)
  discriminator.trainable=True
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
  discriminator.trainable=False
  generator = create_generator(shape,all, dim)
  gan = create_gan(generator, discriminator)
  gan.compile(loss='hinge', optimizer=optimizer,metrics=['accuracy'])
  x_train = np.expand_dims(x_train, axis=3)
#ラベルの作成
  real=[]
  fake=[]
  for i in range(batch_size):
    real.append([0,1])
    fake.append([1,0])
  real=np.array(real)
  fake=np.array(fake)
#学習
  for iteration in range(iterations):
#訓練データをランダムに取り出す
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    words = x_train[idx]
#生成器に入力するランダムなベクトルz
    z = np.random.normal(0, 1, (batch_size, dim))
#生成器が生成した偽単語gen_words
    gen_words = generator.predict(z)
#d_loss_fは判定器の偽単語に対する学習での誤差と正解率の平均
    d_loss_f = discriminator.train_on_batch(gen_words, fake)
#d_loss_rは判定器の本物の単語に対する学習での誤差と正解率の平均
    d_loss_r = discriminator.train_on_batch(words, real)
#誤差と正解率の全体平均を取る
    d_loss, acc = 0.5 * np.add(d_loss_r, d_loss_f)
#生成器に入力するランダムなベクトルz
    z = np.random.normal(0, 1, (batch_size, 100))
    
#g_lossは生成器の学習（生成器としては判定器に本物と判定されるのが正解なので、ラベルはreal）
    gen_words = generator.predict(z)
    g_loss = gan.train_on_batch(z, real)

#sample_intervalごとに損失値と精度を保存
    if (iteration+1) % sample_interval == 0:
      print(f"iteration:{iteration+1}")
      print(f"loss:{d_loss},{g_loss[1]} accuracy:{acc}")
      loss_m.append((d_loss, g_loss[1]))
      acc_m.append(acc)
      test_loss, test_acc = discriminator.evaluate(x_test, y_test, verbose=0)
      print(f"testloss:{test_loss}")
      predictions = discriminator.predict(x_test)
      print(predictions)

#セーブ
  generator.save('dcgan_generator.h5')
  discriminator.save('dcgan_discriminator.h5')
  gan.save('dcgan.h5')
  return generator,discriminator,gan,loss_m,acc_m
generator,discriminator,gan,loss_m,acc_m=train(x_train,(5,26),130,100,'adam',batch_size=1000, sample_interval=10,iterations=1000)

iteration:10
loss:0.5978239476680756,0.0 accuracy:0.5
testloss:0.685966968536377
[[0.5764383  0.44381183]
 [0.56602156 0.4444792 ]
 [0.5862113  0.43183133]
 [0.57844925 0.4299243 ]
 [0.63826483 0.39897895]
 [0.5516421  0.41144615]]
iteration:20
loss:0.5946579277515411,0.0 accuracy:0.5
testloss:0.6583385467529297
[[0.55341995 0.46183103]
 [0.55174905 0.45195252]
 [0.58155954 0.42979732]
 [0.5916777  0.4139027 ]
 [0.6764781  0.35540324]
 [0.55555105 0.4127382 ]]
iteration:30
loss:0.6456539332866669,0.0 accuracy:0.5
testloss:0.6358963251113892
[[0.5354357  0.47380206]
 [0.54839873 0.4454617 ]
 [0.5789881  0.42491278]
 [0.620443   0.37307274]
 [0.6989542  0.32799944]
 [0.56125    0.40254015]]
iteration:40
loss:0.6356931328773499,0.0 accuracy:0.5
testloss:0.6643498539924622
[[0.5538842  0.45508376]
 [0.5818525  0.4095947 ]
 [0.56938285 0.43532646]
 [0.61844534 0.3712532 ]
 [0.6522178  0.3859549 ]
 [0.57089967 0.4048045 ]]
iteration:50
loss:0.6622134149074554,0.003000000026077032 accuracy:0.

In [49]:
def generation(size=30,dim=100):
  z = np.random.normal(0, 1, (size, dim))
  gen_words = generator.predict(z)
  prediction = discriminator.predict(gen_words)
  for i in range(size):
    print(convert(gen_words[i]))
generation()

deems
beioy
loien
craoy
buncn
drfes
foule
telie
pians
berms
beehe
duecr
beneg
bueen
beres
lrlls
taiet
tauze
auiat
lousx
fnaoy
beeen
cailt
benen
burhs
pjeas
foatt
sagnt
dreus
anleg
